In [ ]:
import matplotlib

# Configuramos el uso de TkAgg como backend para matplotlib
matplotlib.use("TkAgg")
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter

In [ ]:
# Definimos una función personalizada para mostrar imágenes
def imshow(
    img,
    new_fig=True,
    title=None,
    color_img=False,
    blocking=False,
    colorbar=False,
    ticks=False,
):
    # Si new_fig es True, crea una nueva figura
    if new_fig:
        plt.figure()
    # Si color_img es True, muestra la imagen en su color original
    if color_img:
        plt.imshow(img)
    else:
        # Si no, muestra la imagen en escala de grises
        plt.imshow(img, cmap="gray")
    # Agregar título a la imagen
    plt.title(title)
    # Si no se desea mostrar los ticks, se eliminan
    if not ticks:
        plt.xticks([]), plt.yticks([])
    # Si se pide, añadir una barra de colores
    if colorbar:
        plt.colorbar()
    # Mostrar la imagen con la opción de bloqueo si es necesario
    if new_fig:
        plt.show(block=blocking)

In [ ]:
# Función para realizar la reconstrucción morfológica de una imagen
def imreconstruct(marker, mask, kernel=None):
    # Asegurarse de que 'marker' y 'mask' tengan el mismo tamaño y tipo
    if marker.shape != mask.shape:
        raise ValueError("El tamaño de 'marker' y 'mask' debe ser igual")
    if marker.dtype != mask.dtype:
        marker = marker.astype(mask.dtype)

    # Si no se proporciona un kernel, utilizar uno por defecto (3x3)
    if kernel is None:
        kernel = np.ones((3, 3), np.uint8)

    while True:
        # Dilatación del marcador
        expanded = cv2.dilate(marker, kernel)

        # Realizar una intersección entre la imagen dilatada y la máscara
        expanded_intersection = cv2.bitwise_and(expanded, mask)

        # Verificar si la reconstrucción ha convergido (si no hay cambios)
        if np.array_equal(marker, expanded_intersection):
            break

        # Actualizar el marcador para la siguiente iteración
        marker = expanded_intersection

    return expanded_intersection

La función imreconstruct implementa el algoritmo de reconstrucción morfológica para imágenes. Este proceso es fundamental en la segmentación y el análisis de imágenes, permitiendo extraer o restaurar estructuras específicas dentro de una imagen. En términos generales, la reconstrucción morfológica se utiliza para propagar las regiones definidas en una imagen inicial (marcador) dentro de los límites impuestos por otra imagen (máscara).

- Entrada:

    - marker: Imagen inicial que define las regiones de inicio.
    - mask: Imagen que establece los límites de propagación.
    - kernel (opcional): Estructura que define la vecindad para la operación de dilatación.

- Salida:
    - Imagen reconstruida, donde las regiones del marcador se expanden dentro de los límites de la máscara según las operaciones morfológicas.

El algoritmo itera aplicando dilataciones sobre la imagen marcador, seguida de una intersección con la máscara, hasta alcanzar un estado de convergencia donde no se producen cambios.

La lógica iterativa garantiza que el proceso de reconstrucción sea robusto y adaptable a imágenes de diferentes tipos y resoluciones. Además, incluir el parámetro opcional kernel proporciona flexibilidad para abordar tareas específicas de procesamiento, desde ajustes finos en microestructuras hasta operaciones en imágenes más grandes.

- Precisión en la segmentación: Este método permite una segmentación más precisa, ya que las regiones se expanden controladamente dentro de los límites definidos por la máscara.
- Control personalizado: La función admite un kernel configurable, lo que permite adaptar el proceso a diferentes requisitos de vecindad en imágenes.
- Versatilidad: Puede usarse en múltiples aplicaciones como:
    - Eliminación de ruido.
    - Restauración de objetos conectados.
    - Mejoramiento de bordes o formas específicas.
- Optimización iterativa: La reconstrucción converge automáticamente cuando no hay cambios en la imagen, evitando iteraciones innecesarias.

In [ ]:
# Función para rellenar huecos en una imagen binaria
def imfillhole(img):
    # Crear una máscara de ceros para los bordes de la imagen
    mask = np.zeros_like(img)
    # Generar bordes alrededor de la máscara para realizar la operación de dilatación
    mask = cv2.copyMakeBorder(
        mask[1:-1, 1:-1], 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=int(255)
    )
    # El marcador es el complemento de los bordes
    marker = cv2.bitwise_not(img, mask=mask)
    # La máscara es el complemento de la imagen original
    img_c = cv2.bitwise_not(img)
    # Realizar la reconstrucción morfológica
    img_r = imreconstruct(marker=marker, mask=img_c)
    # La imagen con los huecos rellenos es el complemento de la reconstrucción
    img_fh = cv2.bitwise_not(img_r)
    return img_fh


La función imfillhole se encarga de rellenar huecos en una imagen binaria, un proceso crucial en el análisis de imágenes para asegurar la continuidad de objetos o regiones. Los huecos se definen como áreas negras rodeadas por regiones blancas en una imagen binaria. Este método garantiza que dichas áreas sean identificadas y completadas adecuadamente.

- Entrada:

    - img: Imagen binaria en la que se desea rellenar los huecos.
- Salida:

    - Imagen binaria procesada donde los huecos han sido rellenados.

El método aprovecha la reconstrucción morfológica para identificar y completar los huecos:

- Se genera una máscara a partir del borde exterior de la imagen.
- El marcador inicial se construye como el complemento de la imagen dentro de la máscara.
- La reconstrucción morfológica propaga las regiones a partir del marcador, limitándose por la máscara.
- Finalmente, se devuelve el complemento de la imagen reconstruida para obtener la imagen final con los huecos rellenados.

Además, al estar basada en la función imreconstruct, tu código aprovecha la flexibilidad y modularidad de esta implementación, permitiendo una integración fluida con otros pasos del pipeline de procesamiento. Esto hace que sea un enfoque eficiente y personalizable para tareas complejas.

Esta función es esencial en contextos donde los objetos deben representarse como regiones completas sin interrupciones. Ejemplos prácticos incluyen:

- Preparación para segmentación: Asegura que los objetos estén completamente conectados antes de aplicar algoritmos de segmentación o clasificación.
- Post-procesamiento en visión computacional: Mejora la calidad de las imágenes binarizadas para análisis como reconocimiento de formas o mediciones precisas.
- Eliminación de imperfecciones: Rellena artefactos no deseados que aparecen como huecos debido a ruido o fallos en etapas previas de procesamiento.

In [ ]:
# Función para encontrar el menor factor de forma
def menor_factor_de_forma(factor):
    primero = 1  # Valor inicial más grande para el factor de forma
    indiceuno = 0
    for registro in factor:
        if registro[1] < primero:  # Si encontramos un valor más pequeño
            primero = registro[1]
            indiceuno = registro[0]
    factor.pop(indiceuno - 1)  # Eliminar el elemento con el factor más pequeño
    return indiceuno, primero, factor

Funcionalidad
La función menor_factor_de_forma identifica el menor factor de forma dentro de una lista de registros y devuelve la información asociada al mismo. Esta operación es útil en análisis donde es necesario priorizar elementos con características mínimas específicas, como en análisis de formas o geometría de objetos.

- Entrada:

    - factor: Lista de tuplas o listas, donde cada elemento contiene un índice y un valor asociado al factor de forma. Ejemplo: [(1, 0.8), (2, 0.5), (3, 0.9)].
- Salida:

    - Índice (indiceuno) correspondiente al menor factor de forma.
    - Valor del menor factor de forma (primero).
    - Lista actualizada con el elemento correspondiente eliminado.
El algoritmo recorre la lista buscando el menor valor en la segunda posición de cada registro y elimina ese elemento de la lista original, devolviendo la información necesaria.

Ventajas
- Simplicidad y claridad: La función es directa en su propósito, lo que facilita su comprensión e integración en pipelines más complejos.
- Optimización del análisis: Identifica y elimina el menor factor en una sola pasada, reduciendo la necesidad de operaciones redundantes.
- Versatilidad: Puede adaptarse para trabajar con diferentes métricas, no solo factores de forma, ampliando su aplicabilidad.
- Estructura ordenada: Devuelve los resultados de forma estructurada, permitiendo un uso inmediato en cálculos posteriores.


Justificación de Uso
Este enfoque es ideal para escenarios donde es necesario iterar sobre una lista priorizando elementos según un criterio de minimización, como:

- Selección óptima en análisis geométrico: Identificar objetos con características específicas (menor asimetría, menor compactación, etc.).
- Procesamiento iterativo en clustering: Reducir iterativamente una lista de candidatos con el menor costo o factor asociado.
- Sistemas de optimización y clasificación: Usar el menor valor como pivote para decisiones posteriores.
- La eliminación directa del menor factor asegura que la lista se reduce dinámicamente, simplificando iteraciones futuras y mejorando la eficiencia computacional.



In [ ]:
# Función para encontrar el área más pequeña en una lista de monedas
def menor_area(monedas):
    area_chica = 100000000  # Valor inicial muy grande para el área
    for i in range(len(monedas)):
        if monedas[i][1] < area_chica:  # Si encontramos un área más pequeña
            area_chica = monedas[i][1]
            indice = i
    return indice

Funcionalidad
La función menor_area busca encontrar el índice de la moneda con el área más pequeña dentro de una lista. Este tipo de análisis es común en procesamiento de imágenes y visión por computadora, especialmente cuando se trabaja con objetos detectados o segmentados, como monedas, partículas, o regiones de interés.

- Entrada:

    - monedas: Lista donde cada elemento representa una moneda. Se espera que cada elemento sea una tupla o lista con al menos dos valores:
        - Identificador o índice de la moneda.
        - Área de la moneda.
- Salida:

    - indice: Índice de la moneda con el área más pequeña.


El algoritmo recorre toda la lista y compara cada área con un valor inicial muy grande (area_chica). Si encuentra un área menor, actualiza tanto el valor del área mínima como el índice correspondiente. Finalmente, devuelve el índice del elemento con el área más pequeña.

Ventajas
- Simplicidad en el diseño: La función es fácil de entender y directa en su propósito, ideal para integrarse en sistemas de procesamiento más complejos.
- Aplicación específica: Es útil para análisis donde se necesita priorizar elementos pequeños, como:
    - Eliminación de artefactos o ruido basado en tamaño.
    - Identificación de la moneda más pequeña en una imagen para tareas de clasificación o comparación.
- Eficiencia lineal: El recorrido de la lista tiene complejidad 𝑂(𝑛). lo que es eficiente para listas de tamaño moderado.


Justificación de Uso
Esta función puede justificarse en el contexto de procesamiento de imágenes como una herramienta clave para análisis de objetos detectados, como monedas u otras formas similares. Algunas aplicaciones prácticas incluyen:

- Clasificación de objetos según tamaño: Identificar el objeto más pequeño como parte de un proceso de etiquetado o caracterización.
- Filtrado por área mínima: Ayudar a eliminar regiones no deseadas en imágenes segmentadas, como artefactos o ruido.
- Optimización en sistemas de conteo: Localizar elementos pequeños puede ser crucial en sistemas que evalúan la calidad o clasifican objetos.


El uso de un valor inicial alto garantiza que cualquier área de la lista será seleccionada, incluso en caso de valores extremadamente pequeños. Esto hace que la función sea robusta para trabajar con datos variados y asegura que siempre haya un resultado.



In [ ]:
# Leer la imagen de monedas
moneda = cv2.imread("monedas.jpg", cv2.IMREAD_COLOR)
# Convertir la imagen de BGR a RGB
moneda_original = cv2.cvtColor(moneda, cv2.COLOR_BGR2RGB)
# Convertir la imagen a escala de grises
img_fil_gray = cv2.cvtColor(moneda_original, cv2.COLOR_RGB2GRAY)
# Aplicar un filtro gaussiano para suavizar la imagen
filtrado = cv2.GaussianBlur(img_fil_gray, (5, 5), 0)
# Detectar bordes utilizando el algoritmo de Canny
canny = cv2.Canny(filtrado, 75, 150)
# Aplicar dilatación a la imagen de bordes para agrandar los objetos
dilatacion = cv2.dilate(canny, np.ones((13, 13), np.uint8))
# Realizar una operación de cierre morfológico para rellenar huecos
img_modif = cv2.morphologyEx(dilatacion, cv2.MORPH_CLOSE, np.ones((27, 27), np.uint8))


imshow(dilatacion, title="Original")  # Descomentar si deseas ver la dilatación

Funcionalidad
Este código realiza un preprocesamiento sobre una imagen de monedas para preparar los datos y facilitar su análisis posterior. El flujo de trabajo incluye pasos clave como la conversión de color, filtrado, detección de bordes y operaciones morfológicas. El objetivo es realzar las características relevantes de las monedas, asegurando una segmentación más precisa.

- Lectura y conversión de la imagen:

    - Se carga la imagen monedas.jpg en color y se convierte a RGB para estandarizar su formato.
    - Luego, se convierte a escala de grises (GRAY) para simplificar las operaciones posteriores.
- Filtrado y detección de bordes:

    - Se aplica un filtro gaussiano para suavizar la imagen y reducir el ruido, lo que mejora la detección de bordes.
    - Con el algoritmo de Canny, se detectan los bordes principales de los objetos (monedas) en la imagen.
- Operaciones morfológicas:

    - La dilatación expande los bordes detectados, asegurando que las monedas tengan contornos más definidos.
    - El cierre morfológico rellena huecos dentro de los objetos, convirtiendo bordes incompletos en regiones sólidas y conectadas.
- Visualización:
    - la línea imshow permite visualizar la imagen dilatada para inspeccionar los resultados intermedios.

Ventajas
- Preparación robusta de datos: Este pipeline combina técnicas que aseguran que las monedas estén bien definidas y listas para el análisis posterior.
- Reducción de ruido: El filtro gaussiano y las operaciones morfológicas mitigan los efectos del ruido, mejorando la precisión de detección.
- Generalización: El enfoque es adaptable a imágenes con diferentes condiciones de iluminación o resolución.
- Modularidad: Cada paso del flujo de trabajo es independiente, lo que facilita ajustes específicos según los requisitos de la tarea.

Justificación de Uso
El preprocesamiento es un paso esencial en cualquier pipeline de procesamiento de imágenes. En este caso:

- Filtrado y detección precisa: La combinación de filtrado gaussiano y Canny asegura que solo se detecten bordes relevantes, reduciendo artefactos o detalles irrelevantes.
- Mejoramiento de contornos: La dilatación y el cierre garantizan que las monedas sean tratadas como regiones completas, lo que facilita análisis como el cálculo de áreas o identificación de factores de forma.
- Estandarización para segmentación: La imagen procesada tiene características uniformes que permiten una segmentación y análisis consistente, incluso en conjuntos de datos variados.
- Este pipeline es ideal para proyectos donde el objetivo es extraer información de objetos circulares o aislados en una imagen, como la identificación de monedas, partículas o células.

In [ ]:
# Llenar los huecos en la imagen de las monedas
relleno = imfillhole(img_modif)

imshow(relleno, title="Moneda Rellenada")  # Descomentar si deseas ver la imagen rellena

In [ ]:
# Aplicar una operación de erosión a la imagen de las monedas rellenas
erocion = cv2.erode(relleno, np.ones((41, 41), np.uint8))

imshow(erocion, title="Monedas Erocionadas")  # Descomentar si deseas ver la erosión

In [ ]:
# Inicializar las listas para el factor de forma y las cajas
factordeforma = []
caja = []
# Obtener los componentes conectados en la imagen de las monedas erosionadas
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(erocion)
for i_label in range(1, num_labels):
    area = stats[i_label, cv2.CC_STAT_AREA]
    filtered_labels = np.zeros_like(erocion, dtype=np.uint8)
    filtered_labels[labels == i_label] = 255
    # Encontrar los contornos de los componentes
    contours, _ = cv2.findContours(
        filtered_labels, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE
    )
    for contour in contours:
        # Calcular el bounding box (caja delimitadora)
        x, y, w, h = cv2.boundingRect(contour)
        perimetro = cv2.arcLength(contour, True)  # Perímetro del contorno
        factor_forma = area / (perimetro**2)  # Calcular el factor de forma
        factordeforma.append([i_label, factor_forma])  # Guardar factor de forma
        caja.append([i_label, area, x, y, w, h])  # Guardar información de la caja

# Encontrar el menor factor de forma (más pequeño)
indice, factor, factordeforma2 = menor_factor_de_forma(factordeforma)
indice2, factor2, factordeforma3 = menor_factor_de_forma(factordeforma2)


Funcionalidad
Este segmento de código realiza la análisis de componentes conectados en la imagen procesada para identificar y analizar objetos individuales (en este caso, monedas). Posteriormente, calcula métricas geométricas como el factor de forma y extrae las cajas delimitadoras de cada objeto.

- Inicialización de listas:

    - factordeforma: Almacena el índice del componente conectado y su respectivo factor de forma.
    - caja: Almacena información geométrica clave para cada objeto (área y bounding box).
- Detección de componentes conectados:

    - cv2.connectedComponentsWithStats: Identifica regiones conectadas (objetos) en la imagen erosionada (erocion) y devuelve:
    - num_labels: Cantidad de componentes detectados.
    - labels: Imagen etiquetada donde cada píxel pertenece a un componente.
    - stats: Métricas estadísticas de cada componente (como área, bounding box, etc.).
    - centroids: Coordenadas de los centroides de cada componente.
- Análisis por componente: Para cada componente identificado:

- Se calcula el área y se crea una máscara binaria específica para ese componente.
- Se encuentran los contornos del objeto dentro de la máscara (cv2.findContours).
- Se calcula:
    - Bounding box (x, y, w, h): Define el rectángulo mínimo que contiene el objeto.
    - Perímetro (cv2.arcLength): Longitud del contorno.
    - Factor de forma: Relación entre el área y el cuadrado del perímetro, utilizada como métrica de compactación o regularidad.
    - Estos valores se almacenan en las listas factordeforma y caja.

- Identificación de factores de forma mínimos:

    - Se utiliza la función menor_factor_de_forma para encontrar los componentes con los menores factores de forma.
    - Esto ayuda a identificar los objetos menos compactos o con mayor irregularidad.

Ventajas
- Análisis completo: Combina la detección de componentes conectados con métricas geométricas avanzadas.
- Automatización: Permite calcular automáticamente el factor de forma y extraer bounding boxes para múltiples objetos.
- Flexibilidad: El enfoque es modular y puede adaptarse a diferentes tipos de imágenes y métricas.
- Identificación de características clave: La métrica de factor de forma es útil para priorizar objetos según su geometría (como círculos, elipses o formas más complejas).

Justificación de Uso
Este enfoque es crucial en contextos donde se deben analizar múltiples objetos segmentados en una imagen. En el caso específico de monedas:

- Caracterización geométrica: El factor de forma ayuda a distinguir entre monedas regulares (compactas) y artefactos o ruido (menos compactos).
- Detección y extracción de regiones: Las cajas delimitadoras permiten aislar monedas individuales para análisis o procesamiento adicional.
- Filtrado basado en propiedades: Identificar los factores de forma más bajos permite enfocar el análisis en los objetos más irregulares, lo cual es útil para detectar anomalías.

La combinación de componentes conectados, contornos y bounding boxes proporciona una solución robusta para análisis geométricos en imágenes complejas.

En este caso, el factor de forma se refiere a una métrica utilizada para evaluar la "compactitud" o la eficiencia geométrica de una figura, como una moneda o un dado. Es una relación entre el área de la figura y su perímetro

El factor de forma ayuda a medir cuánto se aleja una figura de una forma ideal (por ejemplo, un círculo para monedas o un cuadrado para dados). Cuanto más cercano sea el factor a 1, más "compacta" o similar a una forma geométrica regular es la figura.

In [ ]:
# Seleccionar las monedas con los menores factores de forma
monedas = []
for i in range(len(factordeforma3)):
    for j in range(len(caja)):
        if factordeforma3[i][0] == caja[j][0]:
            monedas.append(caja[j])
            continue

Este fragmento selecciona las monedas correspondientes a los índices con los menores factores de forma calculados previamente y las almacena en la lista monedas. El propósito es filtrar y priorizar objetos específicos en función de sus características geométricas, como la compactación.

- Recorrido de las listas:
    - Para cada elemento en factordeforma3 (que contiene los índices y factores de forma más pequeños):
        - Se busca un elemento correspondiente en la lista caja (que contiene información completa de cada componente, como área y bounding box).
        - Si se encuentra una coincidencia (el índice en ambas listas es igual), se agrega esa caja a la lista monedas.
- Almacenamiento en monedas:

    - La lista resultante contiene únicamente las cajas delimitadoras y propiedades geométricas de los objetos con menor factor de forma.

Este enfoque combina métrica geométrica (factor de forma) con segmentación por componentes conectados, mostrando cómo se pueden aislar objetos relevantes de forma eficiente. Esto evidencia la capacidad de tu código para adaptarse a análisis específicos en conjuntos complejos de datos.

In [ ]:
# Seleccionar las cajas de dados
dados = []
for i in range(len(caja)):
    if caja[i][0] == indice:
        dados.append(caja[i])
    elif caja[i][0] == indice2:
        dados.append(caja[i])

Este fragmento de código selecciona las cajas correspondientes a los índices de los dos objetos con menor factor de forma (ya calculados anteriormente como indice e indice2) y las almacena en la lista dados. El propósito es aislar estos objetos, que podrían representar formas específicas dentro de la imagen (en este caso, podrían ser los "dados" u objetos con las características geométricas más inusuales).

Funciona de manera similar a Monedas



In [ ]:
# Definir los umbrales para las monedas
umbrales = {
    '10 centavos': (0, 50000),
    '50 centavos': (70001, 100000),
    '1 peso': (50001, 70000)
}

# Listas iniciales para almacenar monedas etiquetadas
monedas_etiqueta = []

# Iterar sobre cada moneda
for moneda in monedas:
    area = moneda[1]  # Segundo elemento: área
    etiqueta = 'No definido'
    for nombre, (min_area, max_area) in umbrales.items():
        if min_area <= area <= max_area:
            etiqueta = nombre
            break
    monedas_etiqueta.append([etiqueta, moneda[2], moneda[3], moneda[4], moneda[5]])

# Ordenar las monedas etiquetadas según la categoría
monedas_etiqueta = sorted(monedas_etiqueta, key=lambda x: x[0])

In [ ]:
monedas_etiqueta

In [ ]:
monedas_etiqueta

In [ ]:
monedas

Este fragmento de código selecciona las monedas más pequeñas de una lista específica de objetos (previamente extraída) y las etiqueta según su valor nominal (10 centavos, 1 peso, 50 centavos). Posteriormente, las monedas seleccionadas son almacenadas junto con su información de coordenadas (bounding box) para diferenciarlas según su denominación.

- Selección de monedas más pequeñas:

    - Se utiliza la función menor_area(monedas) para seleccionar el índice correspondiente a la moneda con el área más pequeña dentro de la lista monedas.
        - Esto asegura que solo las monedas más pequeñas sean consideradas.
- Etiquetado y agrupamiento:

    - Dependiendo del rango de iteración (0-8, 9-13, 14-16), se etiquetan las monedas según su denominación:
        - 10 centavos: Para los primeros 9 objetos.
        - 1 peso: Para los siguientes 5 objetos (índices 9 a 13).
        - 50 centavos: Para los últimos 3 objetos (índices 14 a 16).
    Cada moneda seleccionada junto con su información de coordenadas (x, y, w, h) es almacenada en la lista monedas_etiqueta.
- Eliminación del objeto procesado:

    - Finalmente, la moneda seleccionada (basada en el área mínima) es eliminada de la lista monedas para evitar repetir el proceso.

In [ ]:
# Realizar dilatación y seleccionar los dados
delatacion_copia = np.zeros_like(dilatacion, dtype=np.uint8)
delatacion_copia[dados[0][3] : dados[0][3] + dados[0][5], dados[0][2] : dados[0][2] + dados[0][4]] = dilatacion[dados[0][3] : dados[0][3] + dados[0][5], dados[0][2] : dados[0][2] + dados[0][4]]
delatacion_copia[dados[1][3] - 20 : dados[1][3] + dados[1][5],dados[1][2] : dados[1][2] + dados[1][4],] = dilatacion[dados[1][3] - 20 : dados[1][3] + dados[1][5],dados[1][2] : dados[1][2] + dados[1][4],]

imshow(delatacion_copia, title="Dados")  # Descomentar si deseas ver los dados

Este fragmento de código realiza una dilatación selectiva en la imagen de bordes (dilatacion) para aislar dos dados seleccionados previamente (dados). El objetivo es expandir las áreas de interés (bounding boxes de los dados) para obtener resultados más claros y detallados.

- Creación de una copia de la imagen de dilatación:

    - delatacion_copia es una máscara inicial que comienza como una imagen en blanco (np.zeros_like) con el mismo tamaño que dilatacion.
- Dilatación selectiva:

    - La dilatación se realiza solo en las coordenadas correspondientes a los dados seleccionados:
        -   Para el primer dado: Se toma un área alrededor de su bounding box (dados[0]), pero con una expansión adicional.
        - Para el segundo dado: Se toma su región, aplicando también una expansión de 20 píxeles para cada lado.
- Integración de resultados:

    - Ambas áreas de dilatación se combinan en delatacion_copia utilizando las coordenadas exactas de los dados.

In [ ]:
# Erosionar los dados para mejorar la detección
erocion_dados = cv2.erode(delatacion_copia, np.ones((11, 11), np.uint8))
relleno_dado = imfillhole(erocion_dados)

imshow(relleno_dado, title="Dados Rellenados")  # Descomentar si deseas ver los dados rellenados

# Erosionar nuevamente
erocion_dados2 = cv2.erode(relleno_dado, np.ones((11, 11), np.uint8))

imshow(erocion_dados2, title="Dados Erocionados")  # Descomentar si deseas ver los dados erosionados

Este fragmento de código realiza operaciones de erosión y relleno de huecos para mejorar la detección y la limpieza de los dados seleccionados previamente. La finalidad es mejorar la precisión en la identificación y separación de objetos tras las operaciones de dilatación.

In [ ]:
# Obtener los componentes conectados en la imagen de los dados
num_labels_dado, labels_dado, stats_dado, centroids_dado = (
    cv2.connectedComponentsWithStats(erocion_dados2)
)
foto = np.zeros_like(relleno_dado, dtype=np.uint8)
for i_label in range(1, num_labels_dado):
    area_dado = stats_dado[i_label, cv2.CC_STAT_AREA]
    if area_dado > 1000:
        foto[labels_dado == i_label] = 255
        
imshow(foto, title="Dados")  # Descomentar si deseas ver los dados

Este fragmento de código identifica y procesa los componentes conectados en la imagen después de aplicar la erosión para los dados seleccionados. Los objetos son extraídos y evaluados según su área para clasificar los dados en función de su tamaño.

- Componentes conectados:

    - cv2.connectedComponentsWithStats(erocion_dados2) calcula los componentes conectados en la imagen erocion_dados2.
    - num_labels_dado almacena la cantidad de componentes, labels_dado identifica cada componente, y stats_dado almacena las estadísticas asociadas a cada componente (como área y coordenadas).
- Filtrado por área:

    - Sólo los componentes cuyo área (area_dado) es mayor que 1000 son considerados. Esto ayuda a excluir componentes no deseados que puedan ser demasiado pequeños o irrelevantes.
- Creación de la imagen procesada:

    - foto es una imagen en blanco que se llena con 255 en los lugares donde se encuentran componentes relevantes.

In [ ]:
# Etiquetar las caras de los dados
dado_etiqueta = []
cara_dado1 = cv2.connectedComponentsWithStats(
    foto[  # Imagen recortada para el primer dado
        dados[0][3] : dados[0][3] + dados[0][5], dados[0][2] : dados[0][2] + dados[0][4]
    ]
)
cara_dado2 = cv2.connectedComponentsWithStats(
    foto[  # Imagen recortada para el segundo dado
        dados[1][3] : dados[1][3] + dados[1][5], dados[1][2] : dados[1][2] + dados[1][4]
    ]
)

imshow(foto,title="Dados")  # Descomentar si deseas ver los dados

Este fragmento de código realiza la identificación y etiquetado de las caras de los dados, utilizando conectividad conectada para cada dado seleccionado. Las imágenes recortadas se utilizan para procesar cada dado individualmente y obtener las caras visibles.

In [ ]:
# Guardar las etiquetas para los dados
dado_etiqueta.append(
    [
        f"Valor de la cara = {cara_dado1[0]-1}",
        dados[0][2],
        dados[0][3],
        dados[0][4],
        dados[0][5],
    ]
)
dado_etiqueta.append(
    [
        f"Valor de la cara = {cara_dado2[0]-1}",
        dados[1][2],
        dados[1][3],
        dados[1][4],
        dados[1][5],
    ]
)

In [ ]:
copia = moneda_original.copy()
for etiqueta, x, y, ancho, alto in monedas_etiqueta:
    # Coordenadas del rectángulo
    punto1 = (x, y)  # Esquina superior izquierda
    punto2 = (x + ancho, y + alto)  # Esquina inferior derecha

    # Dibujar el rectángulo
    cv2.rectangle(copia, punto1, punto2, color=(0, 255, 0), thickness=10)

    # Añadir la etiqueta (texto)
    cv2.putText(copia,etiqueta,(x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX,fontScale=2,color=(0, 255, 0),thickness=10,)

imshow(copia, title="Monedas Etiquetadas")  # Descomentar si deseas ver las monedas etiquetadas


Este fragmento de código realiza la visualización de las monedas etiquetadas dibujando rectángulos alrededor de cada moneda en la imagen original. Además, se añade una etiqueta con texto para identificar cada moneda.

In [24]:
for etiqueta, x, y, ancho, alto in dado_etiqueta:
    # Coordenadas del rectángulo
    punto1 = (x, y)  # Esquina superior izquierda
    punto2 = (x + ancho, y + alto)  # Esquina inferior derecha

    # Dibujar el rectángulo
    cv2.rectangle(copia, punto1, punto2, color=(0, 255, 0), thickness=10)

    # Añadir la etiqueta (texto)
    cv2.putText(copia,etiqueta,(x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX,fontScale=2,color=(0, 255, 0),thickness=10,)

imshow(copia, title="Dados y monedas etiquetados por valor", blocking=True)